# YouTube Report Generator - Phase 2 Full Pipeline (Improved)

## 📋 Overview

**Version**: 2.2 (Enhanced with prompt tuning and analysis improvements)

### Key Improvements in v2.2
- ✅ **Two-Part Summary Structure**: KEY POINTS (1-2 sentences) + DETAILED SUMMARY (3-5 sentences)
- ✅ **Enhanced Comment Analysis**: Majority/Minority views + Sentiment breakdown
- ✅ **LLM Category Correction**: Validates and corrects category model predictions
- ✅ **Increased Comment Processing**: Up to 100 comments (configurable)
- ✅ **Better Report Formatting**: Structured sections for improved readability

### Base Features from v2.1
- ✅ **Multilingual Support**: Llama/Qwen native multilingual understanding
- ✅ **Fixed English Output**: All reports in English regardless of input language
- ✅ **Code-switching Handling**: Understands mixed-language content naturally
- ✅ **Modular Design**: 모델, 프롬프트, 전처리 모듈화
- ✅ **Extensible Architecture**: Team model 통합 준비
- ✅ **Better Preprocessing**: URL 제거, 언어 감지

---

## 📦 1. Setup & Installation

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install required packages
!pip install -q transformers accelerate bitsandbytes torch
!pip install -q langdetect isodate
!pip install -q google-api-python-client  # For YouTube API (optional)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
import json
import re
import warnings
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field

import torch
import isodate
from langdetect import detect, LangDetectException
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA device: NVIDIA L4


## ⚙️ 2. Configuration

모든 설정을 여기서 제어할 수 있습니다.

In [9]:
@dataclass
class PipelineConfig:
    """전체 파이프라인 설정 (Enhanced for v2.2)"""

    # Model Configuration
    model_name: str = "meta-llama/Llama-3.1-8B-Instruct"
    use_4bit: bool = True
    max_new_tokens: int = 512
    temperature: float = 0.7
    top_p: float = 0.9

    # Data Configuration
    use_youtube_api: bool = False
    youtube_api_key: Optional[str] = None
    data_path: str = "full_dataset_20251013_215347.json"

    # Processing Configuration - ENHANCED
    max_description_length: int = 2000
    max_comments_to_process: int = 100  # 🆕 Increased from 50 to 100
    min_comment_length: int = 10
    remove_urls: bool = True
    detect_language: bool = True

    # Multilingual Configuration
    output_language: str = "English"
    multilingual_understanding: bool = True

    # Experimental Configuration
    num_videos_for_test: int = 3
    enable_detailed_logging: bool = True
    log_token_counts: bool = True

    # Team Model Integration - ENHANCED
    use_category_model: bool = False
    use_sentiment_model: bool = False
    category_model_path: Optional[str] = None
    sentiment_model_path: Optional[str] = None
    enable_llm_category_correction: bool = True  # 🆕 LLM-based category validation

    # Output Configuration
    output_format: str = "markdown"
    save_reports: bool = True
    output_dir: str = "reports"

# Create configuration
config = PipelineConfig(
    # model_name="Qwen/Qwen2.5-7B-Instruct",
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    use_4bit=True,
    use_youtube_api=False,
    data_path="/content/drive/MyDrive/25_sch/ML/youtube_test_10132154/full_dataset_20251013_215347.json",
    output_language="English",
    multilingual_understanding=True,
    num_videos_for_test=3,
    enable_detailed_logging=True,
    max_comments_to_process=100,  # 🆕 Process more comments
    enable_llm_category_correction=True  # 🆕 Enable category correction
)


# YouTube Category List (for LLM Category Correction)
# YouTube Category List (11 categories used by team model)
CATEGORY_LIST = [
    "Music",
    "Sports",
    "Travel",
    "Gaming",
    "People & Blogs",
    "Comedy",
    "Entertainment",
    "News & Politics",
    "HowTo & Style",
    "Education",
    "Science & Tech",
]

print("="*60)
print("Pipeline Configuration (v2.2 Enhanced)")
print("="*60)
print(f"Model: {config.model_name}")
print(f"4-bit: {config.use_4bit}")
print(f"Temperature: {config.temperature}")
print(f"Output Language: {config.output_language}")
print(f"Test Videos: {config.num_videos_for_test}")
print(f"Max Comments to Process: {config.max_comments_to_process} (🆕 Increased)")
print(f"LLM Category Correction: {config.enable_llm_category_correction} (🆕)")
print("="*60)

Pipeline Configuration (v2.2 Enhanced)
Model: meta-llama/Llama-3.1-8B-Instruct
4-bit: True
Temperature: 0.7
Output Language: English
Test Videos: 3
Max Comments to Process: 100 (🆕 Increased)
LLM Category Correction: True (🆕)


## 🧩 3. Enhanced Prompt Templates

### 🆕 Key Improvements:
1. **Two-Part Structure**: KEY POINTS (scannable) + DETAILED SUMMARY (comprehensive)
2. **Majority/Minority Analysis**: Distinguishes mainstream vs outlier opinions
3. **Sentiment Integration**: Explicit sentiment breakdown in output format

In [82]:
class PromptTemplates:
    """Enhanced Prompt Templates (v2.2) with Two-Part Structure"""

    # ===== Video Summary Prompts (ENHANCED) =====
    VIDEO_SUMMARY_SYSTEM = """
You are an expert multilingual content analyst specializing in YouTube video analysis.
Your task is to understand content in ANY language (Korean, English, Japanese, or mixed) and create summaries in English.
You have native-level understanding of multiple languages and can capture nuances across different cultures.
Focus on accuracy and avoid hallucinations.

CRITICAL:
- Follow the TWO-PART output format exactly as specified.
- Do NOT include any guidelines, meta comments, or notes in the output.
- Write natural English sentences only.
"""

    VIDEO_SUMMARY_USER = """
Analyze this YouTube video and create a TWO-PART summary in English.

Video Information:
- Title: {title}
- Description: {description}
- Category: {category}
- Duration: {duration} seconds
- Channel: {channel}

Context:
The title and description may be in Korean, English, Japanese, or mixed languages.
Understand the content in its original languages and provide your analysis.

Output Format (FOLLOW EXACTLY):

KEY POINT:
- 1–2 sentences.
- One clear, scannable statement of what this video is about.
- No bullet points, no headings, just plain sentences.

DETAILED SUMMARY:
- 3–4 sentences.
- Describe the main theme, key content highlights, target audience, and purpose of the video.
- If the description is minimal, briefly mention that information is limited, but do NOT overemphasize this.

Write everything in natural English sentences without repeating labels like "KEY POINT" or "DETAILED SUMMARY" in the text.
"""

    # ===== Reaction Summary Prompts (ENHANCED) =====
    REACTION_SUMMARY_SYSTEM = """
You are an expert in multilingual social media sentiment analysis.
You can understand and analyze comments in Korean, English, Japanese, and mixed languages.
Your task is to capture audience reactions across all language communities and summarize in English.

CRITICAL:
- Follow TWO-PART output structure: KEY POINT first, then DETAILED SUMMARY.
- KEY POINT: 1-2 sentences capturing the dominant sentiment/reaction.
- DETAILED SUMMARY: 3-5 sentences analyzing majority/minority views + themes.
- SENTIMENT BREAKDOWN: Final line with exact percentages (sum to 100%).
- Do NOT include guidelines, meta comments, or notes in output.
- Use qualitative phrases in DETAILED SUMMARY ("overwhelmingly positive", "small minority") instead of numbers.
"""

    REACTION_SUMMARY_USER = """
Analyze these YouTube comments and create a TWO-PART summary + sentiment breakdown.

Video title: {title}

Audience Comments ({comment_count} comments, highest-engagement first):
{comments}

Context:
Multilingual comments (Korean, English, Japanese, code-switching).
Ignore spam/jokes/extreme outliers.

Output Format (FOLLOW EXACTLY):

KEY POINT:
- 1-2 sentences.
- Dominant sentiment and general reaction (e.g., "Overwhelmingly positive excitement").

DETAILED SUMMARY:
- 3-5 sentences.
- Majority view (main patterns/themes).
- Notable minority/outlier views (only meaningful ones).
- Common themes across languages.
- Cultural/language differences if present.
- Use qualitative expressions, NOT numbers.

SENTIMENT BREAKDOWN:
- One line: Positive: X%  Negative: Y%  Neutral: Z% (must sum to 100%)
- Do NOT explicitly add any "Note:" or disclaimer sentences about the analysis. These sentences would be provided anyway with stacticly predefined ones.

Write in natural English sentences without repeating section labels in the text.
"""


    # ===== Category Correction Prompt (NEW) =====
    CATEGORY_CORRECTION_SYSTEM = """You are an expert YouTube category classifier with deep understanding of content types.
Your task is to validate model predictions and correct them if clearly wrong.

Available Categories:
1. Music
2. Sports
3. Travel
4. Gaming
5. People & Blogs
6. Comedy
7. Entertainment
8. News & Politics
9. HowTo & Style
10. Education
11. Science & Tech
Guidelines:
- Only correct if the prediction is CLEARLY wrong
- If model is reasonable (even if not perfect), keep the original
- Music videos should be "Music", not "Entertainment" or "People & Blogs"
- Gaming content should be "Gaming", not "Entertainment"
- Educational content should be "Education", not "People & Blogs"""

    CATEGORY_CORRECTION_USER = """Review this category prediction and correct if clearly wrong.

Model Prediction: {model_prediction}

Video Information:
- Title: {title}
- Description: {description}
- Channel: {channel}

Instructions:
1. Read the video information carefully
2. Evaluate if the model prediction is appropriate
3. If CLEARLY wrong, provide the correct category from the list above
4. If reasonable (even if not perfect), respond with "KEEP_ORIGINAL"
5. Output ONLY the category name or "KEEP_ORIGINAL"

Your decision (category name or KEEP_ORIGINAL):"""

    CUSTOM_PROMPT = None

    @classmethod
    def get_video_summary_prompt(cls, video_info: Dict) -> List[Dict]:
        """Generate video summary prompt (enhanced two-part structure)"""
        if cls.CUSTOM_PROMPT:
            return cls.CUSTOM_PROMPT

        return [
            {"role": "system", "content": cls.VIDEO_SUMMARY_SYSTEM},
            {"role": "user", "content": cls.VIDEO_SUMMARY_USER.format(
                title=video_info.get('title', 'N/A'),
                description=video_info.get('description', 'N/A')[:2000],
                category=video_info.get('category_name', 'N/A'),
                duration=video_info.get('duration_seconds', 'N/A'),
                channel=video_info.get('channel_title', 'N/A')
            )}
        ]

    @classmethod
    def get_reaction_summary_prompt(cls, title: str, comments_text: str, comment_count: int) -> List[Dict]:
        return [
            {"role": "system", "content": cls.REACTION_SUMMARY_SYSTEM},
            {
                "role": "user",
                "content": cls.REACTION_SUMMARY_USER.format(
                    title=title,
                    comments=comments_text,
                    comment_count=comment_count,
                ),
            },
        ]

    @classmethod
    def get_category_correction_prompt(cls, model_prediction: str, video_info: Dict) -> List[Dict]:
        """Generate category correction prompt (NEW)"""
        return [
            {"role": "system", "content": cls.CATEGORY_CORRECTION_SYSTEM},
            {"role": "user", "content": cls.CATEGORY_CORRECTION_USER.format(
                model_prediction=model_prediction,
                title=video_info.get('title', 'N/A'),
                description=video_info.get('description', 'N/A')[:500],
                channel=video_info.get('channel_title', 'N/A')
            )}
        ]

print("✅ Enhanced prompt templates loaded (v2.2)")
print("🆕 Two-part summary structure")
print("🆕 Majority/minority opinion analysis")
print("🆕 Sentiment breakdown integration")
print("🆕 Category correction system")

✅ Enhanced prompt templates loaded (v2.2)
🆕 Two-part summary structure
🆕 Majority/minority opinion analysis
🆕 Sentiment breakdown integration
🆕 Category correction system


## 🔧 4. Text Preprocessing

Same as v2.1 - no changes needed

In [42]:
class TextPreprocessor:
    """텍스트 전처리"""

    def __init__(self, config: PipelineConfig):
        self.config = config
        self.url_pattern = re.compile(r'http[s]?://\S+')

    def remove_urls(self, text: str) -> str:
        return self.url_pattern.sub('', text)

    def detect_language(self, text: str) -> str:
        try:
            lang = detect(text)
            lang_map = {'ko': 'Korean', 'en': 'English', 'ja': 'Japanese'}
            return lang_map.get(lang, 'English')
        except:
            return 'English'

    def clean_description(self, description: str) -> str:
        if not description:
            return "No description available."
        if self.config.remove_urls:
            description = self.remove_urls(description)
        description = ' '.join(description.split())
        if len(description) > self.config.max_description_length:
            description = description[:self.config.max_description_length] + "..."
        if len(description.strip()) < 20:
            return "Minimal description available."
        return description

    def filter_comments(self, comments: List[Dict]) -> List[Dict]:
        filtered = [c for c in comments if len(c.get('text', '')) >= self.config.min_comment_length]
        filtered.sort(key=lambda x: x.get('like_count', 0), reverse=True)
        return filtered[:self.config.max_comments_to_process]

    def format_comments_for_prompt(self, comments: List[Dict]) -> str:
        if not comments:
            return "No comments available."
        formatted = []
        for i, comment in enumerate(comments, 1):
            text = comment.get('text', '')
            likes = comment.get('like_count', 0)
            if self.config.remove_urls:
                text = self.remove_urls(text)
            formatted.append(f"{i}. [{likes} likes] {text}")
        return "\n".join(formatted)

    def parse_duration(self, duration_str: str) -> int:
        try:
            duration = isodate.parse_duration(duration_str)
            return int(duration.total_seconds())
        except:
            return 0

preprocessor = TextPreprocessor(config)
print("✅ Text preprocessor initialized")

✅ Text preprocessor initialized


## 🤖 5. Model Loading

Same as v2.1 - no changes needed

In [43]:
# [OPTIONAL]
# Run this cell if model need to be loaded with hugging face login

from huggingface_hub import login
login()  # Token 입력 필요

In [44]:
class ModelManager:
    """LLM 모델 관리"""

    def __init__(self, config: PipelineConfig):
        self.config = config
        self.model = None
        self.tokenizer = None
        self.pipe = None

    def load_model(self):
        print(f"\n🔄 Loading: {self.config.model_name}")
        print(f"⚙️ 4-bit: {self.config.use_4bit}")

        # Tokenizer
        print("Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.config.model_name, trust_remote_code=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token

        # Quantization config
        if self.config.use_4bit:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
        else:
            quantization_config = None

        # Model
        print("Loading model...")
        self.model = AutoModelForCausalLM.from_pretrained(
            self.config.model_name,
            quantization_config=quantization_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16 if not self.config.use_4bit else None
        )

        # Pipeline
        print("Creating pipeline...")
        self.pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=self.config.max_new_tokens,
            temperature=self.config.temperature,
            top_p=self.config.top_p,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id
        )

        print("✅ Model loaded!")
        if self.config.enable_detailed_logging:
            print(f"📊 Model parameters: ~{self.model.num_parameters() / 1e9:.1f}B")

    def generate(self, messages: List[Dict]) -> str:
        """Generate text with explicit chat template and detailed logging"""
        if self.pipe is None:
            raise RuntimeError("Model not loaded. Call load_model() first.")

        try:
            # Apply chat template explicitly (for reproducibility)
            if hasattr(self.tokenizer, 'apply_chat_template'):
                prompt_text = self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                )

                # Log token counts if enabled
                if self.config.log_token_counts:
                    input_tokens = len(self.tokenizer.encode(prompt_text))
                    if self.config.enable_detailed_logging:
                        print(f"      [Input tokens: {input_tokens}]")

                # Generate
                outputs = self.pipe(messages)
            else:
                # Fallback for models without chat template
                outputs = self.pipe(messages)

            # Extract generated text
            generated_text = outputs[0]["generated_text"]

            if isinstance(generated_text, list):
                # Chat format output
                result = generated_text[-1]["content"]
            else:
                # String output
                result = generated_text

            # Log output token count
            if self.config.log_token_counts:
                output_tokens = len(self.tokenizer.encode(result))
                if self.config.enable_detailed_logging:
                    print(f"      [Output tokens: {output_tokens}]")

            return result

        except Exception as e:
            # Enhanced error logging
            print(f"\n❌ Generation Error:")
            print(f"   Error type: {type(e).__name__}")
            print(f"   Error message: {str(e)}")
            print(f"   Messages length: {len(messages)}")

            if self.config.enable_detailed_logging:
                # Log message content for debugging
                total_chars = sum(len(m.get('content', '')) for m in messages)
                print(f"   Total message chars: {total_chars}")
                print(f"   System prompt length: {len(messages[0].get('content', ''))} chars")
                print(f"   User prompt length: {len(messages[1].get('content', ''))} chars")

            raise

model_manager = ModelManager(config)
model_manager.load_model()


🔄 Loading: meta-llama/Llama-3.1-8B-Instruct
⚙️ 4-bit: True
Loading tokenizer...
Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Creating pipeline...
✅ Model loaded!
📊 Model parameters: ~8.0B


## 🔌 6. Team Model Integration (Enhanced)

### 🆕 Added: LLM-based Category Correction

In [45]:
class TeamModelIntegration:
    """팀 모델 통합 인터페이스 (Enhanced with LLM correction)"""

    def __init__(self, config: PipelineConfig, model_manager: 'ModelManager'):
        self.config = config
        self.model_manager = model_manager
        self.category_model = None
        self.sentiment_model = None

    def load_category_model(self):
        if not self.config.use_category_model:
            return
        # TODO: 팀원 구현
        print("⚠️ Category model not implemented yet")

    def load_sentiment_model(self):
        if not self.config.use_sentiment_model:
            return
        # TODO: 팀원 구현
        print("⚠️ Sentiment model not implemented yet")

    def predict_category(self, video_info: Dict) -> Optional[str]:
        """Predict category (from team model)"""
        if not self.config.use_category_model or self.category_model is None:
            return None
        # TODO: 모델 inference
        return None

    def predict_category_with_llm_correction(self, video_info: Dict, model_pred: str) -> Tuple[str, str, bool]:
        """🆕 Validate and correct category prediction using LLM

        Returns:
            Tuple[str, str, bool]: (original_prediction, final_prediction, was_corrected)
        """
        if not self.config.enable_llm_category_correction:
            return model_pred, model_pred, False

        try:
            # Generate correction prompt
            messages = PromptTemplates.get_category_correction_prompt(model_pred, video_info)

            # Get LLM's decision
            llm_decision = self.model_manager.generate(messages).strip()

            # Check if correction was made
            if llm_decision.upper() == "KEEP_ORIGINAL" or llm_decision == model_pred:
                return model_pred, model_pred, False
            else:
                return model_pred, llm_decision, True

        except Exception as e:
            print(f"⚠️ Category correction failed: {e}")
            return model_pred, model_pred, False

    def analyze_sentiment(self, comments: List[Dict]) -> Optional[Dict]:
        """Analyze sentiment (from team model)"""
        if not self.config.use_sentiment_model or self.sentiment_model is None:
            return None
        # TODO: 모델 inference
        return None

# Initialize after model_manager is created
team_models = TeamModelIntegration(config, model_manager)
team_models.load_category_model()
team_models.load_sentiment_model()
print("✅ Team model interface ready (enhanced with LLM correction)")

✅ Team model interface ready (enhanced with LLM correction)


## 📥 7. Data Loading

Same as v2.1 - no changes needed

In [46]:
class DataLoader:
    def __init__(self, config: PipelineConfig):
        self.config = config

    def load_from_file(self, file_path: str) -> List[Dict]:
        print(f"\n📂 Loading: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ Loaded {len(data)} videos")
        return data

    def get_data(self) -> List[Dict]:
        if self.config.use_youtube_api:
            raise NotImplementedError("YouTube API not implemented yet")
        return self.load_from_file(self.config.data_path)

data_loader = DataLoader(config)
print("✅ Data loader ready")

✅ Data loader ready


## 🎬 8. Enhanced Report Generation Pipeline

### 🆕 Key Improvements:
1. Enhanced video summary generation (two-part structure)
2. Enhanced reaction summary generation (majority/minority + sentiment)
3. Improved report formatting (structured sections)
4. Category correction display in reports

In [80]:
class ReportGenerator:
    """Enhanced Report Generator (v2.2)"""

    def __init__(self, config, model_manager, preprocessor, team_models):
        self.config = config
        self.model_manager = model_manager
        self.preprocessor = preprocessor
        self.team_models = team_models

        # Initialize language statistics
        self.language_stats = {
            'video': {},
            'comments': {}
        }
        self.current_video_lang_dist = {}

    def generate_video_summary(self, video_info: Dict) -> Tuple[str, str]:
        """Generate video summary with two-part structure"""
        # Preprocess
        video_info['description'] = self.preprocessor.clean_description(
            video_info.get('description', '')
        )
        video_info['duration_seconds'] = self.preprocessor.parse_duration(
            video_info.get('duration', 'PT0S')
        )

        # Detect language and update statistics
        detected_lang = 'Unknown'
        if self.config.detect_language:
            detected_lang = self.preprocessor.detect_language(
                video_info.get('title', '') + ' ' + video_info.get('description', '')[:500]
            )

            if self.config.enable_detailed_logging:
                print(f"    [Detected input language: {detected_lang}]")

            self.language_stats['video'][detected_lang] = \
                self.language_stats['video'].get(detected_lang, 0) + 1

        # Generate prompt
        messages = PromptTemplates.get_video_summary_prompt(video_info)

        # Generate summary
        try:
            summary = self.model_manager.generate(messages)

            if self.config.enable_detailed_logging:
                print(f"    [Output language: {self.config.output_language}]")

            return summary.strip(), detected_lang
        except Exception as e:
            print(f"⚠️ Video summary generation failed: {e}")
            return "Video summary generation failed.", detected_lang

    def generate_reaction_summary(self, title: str, comments: List[Dict]) -> Tuple[str, str]:
        """🆕 Generate reaction summary with majority/minority analysis and sentiment"""
        # Filter and format comments
        filtered_comments = self.preprocessor.filter_comments(comments)

        if not filtered_comments:
            return "No comments available for analysis.", "N/A"

        comments_text = self.preprocessor.format_comments_for_prompt(filtered_comments)

        # Detect comment language distribution and update statistics
        lang_distribution = "N/A"
        lang_dist = {}

        if self.config.detect_language:
            # Sample up to N comments (config-based)
            sample_size = min(len(filtered_comments), self.config.max_comments_to_process)
            sample_texts = [
                c.get('text', '')[:100] for c in filtered_comments[:sample_size]
            ]
            langs = [self.preprocessor.detect_language(t) for t in sample_texts if t]

            # Count by language
            if langs:
                for lang in langs:
                    lang_dist[lang] = lang_dist.get(lang, 0) + 1

                lang_distribution = ', '.join([f"{k}: {v}" for k, v in lang_dist.items()])

                if self.config.enable_detailed_logging:
                    print(f"    [Comment languages (top {sample_size}): {lang_dist}]")

                # Update comment language statistics
                for lang, count in lang_dist.items():
                    self.language_stats['comments'][lang] = \
                        self.language_stats['comments'].get(lang, 0) + count

        # Store current video's language distribution
        self.current_video_lang_dist = lang_dist

        # Generate prompt with comment count
        messages = PromptTemplates.get_reaction_summary_prompt(
            title,
            comments_text,
            len(filtered_comments)
        )

        # Generate summary
        try:
            summary = self.model_manager.generate(messages)

            if self.config.enable_detailed_logging:
                print(f"    [Output language: {self.config.output_language}]")

            return summary.strip(), lang_distribution
        except Exception as e:
            print(f"⚠️ Reaction summary generation failed: {e}")
            return "Reaction summary generation failed.", lang_distribution

    def calculate_engagement_metrics(self, video_info: Dict) -> Dict:
        """Calculate engagement metrics"""
        views = video_info.get('view_count', 0)
        likes = video_info.get('like_count', 0)
        comments = video_info.get('comment_count', 0)

        engagement_rate = ((likes + comments) / views * 100) if views > 0 else 0
        like_rate = (likes / views * 100) if views > 0 else 0
        comment_rate = (comments / views * 100) if views > 0 else 0

        return {
            'views': views,
            'likes': likes,
            'comments': comments,
            'engagement_rate': round(engagement_rate, 2),
            'like_rate': round(like_rate, 2),
            'comment_rate': round(comment_rate, 2)
        }

    def get_language_statistics(self) -> Dict:
        """Return accumulated language statistics"""
        return self.language_stats

    def format_markdown_report(self, video_data: Dict, video_summary: str,
                               reaction_summary: str, metrics: Dict,
                               team_predictions: Dict = None,
                               detected_languages: Dict = None) -> str:
        """🆕 Enhanced report formatting with structured sections"""
        video_info = video_data['video_info']
        comments = video_data.get('comments', [])

        # Calculate comment statistics
        filtered_for_analysis = self.preprocessor.filter_comments(comments)
        total_comments = len(comments)
        analyzed_comments = len(filtered_for_analysis)
        analysis_percentage = (analyzed_comments / total_comments * 100) if total_comments > 0 else 0

        report = f"""# YouTube Video Report

**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Model**: {self.config.model_name}
**Pipeline Version**: 2.2 (Enhanced Prompts & Analysis)

---

## 📹 Video Information

- **Title**: {video_info.get('title', 'N/A')}
- **Channel**: {video_info.get('channel_title', 'N/A')}
- **Category**: {video_info.get('category_name', 'N/A')}
- **Published**: {video_info.get('published_at', 'N/A')}
- **Duration**: {self.preprocessor.parse_duration(video_info.get('duration', 'PT0S'))} seconds
- **Video ID**: `{video_info.get('video_id', 'N/A')}`
- **URL**: https://www.youtube.com/watch?v={video_info.get('video_id', '')}
"""

        # Add language detection info
        if detected_languages and self.config.enable_detailed_logging:
            report += "\n### 🌍 Detected Languages\n\n"
            if 'video' in detected_languages:
                report += f"- **Video content**: {detected_languages['video']}\n"
            if 'comments' in detected_languages:
                report += f"- **Comments**: {detected_languages['comments']}\n"
            report += f"- **Report output**: {self.config.output_language}\n"

        report += "\n---\n\n"

        report += f"""## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | {metrics['views']:,} |
| Likes | {metrics['likes']:,} |
| Comments | {metrics['comments']:,} |
| Engagement Rate | {metrics['engagement_rate']}% |
| Like Rate | {metrics['like_rate']}% |
| Comment Rate | {metrics['comment_rate']}% |

---

## 📝 Video Summary

{video_summary}

---

## 💬 Audience Reaction Summary

**Comments Analyzed**: {analyzed_comments:,} / {total_comments:,} ({analysis_percentage:.1f}%)

{reaction_summary}

---
"""

        # 🆕 Enhanced team predictions section with category correction
        if team_predictions:
            report += """## 🤖 Team Model Predictions\n\n"""

            if 'category' in team_predictions:
                cat_info = team_predictions['category']
                if isinstance(cat_info, dict):
                    report += f"- **Category Prediction**:\n"
                    report += f"  - Original: {cat_info.get('original', 'N/A')}\n"
                    report += f"  - Final: {cat_info.get('final', 'N/A')}\n"
                    if cat_info.get('corrected', False):
                        report += f"  - Status: ✅ Corrected by LLM\n"
                    else:
                        report += f"  - Status: ✓ Validated by LLM\n"
                else:
                    report += f"- **Predicted Category**: {cat_info}\n"

            if 'sentiment' in team_predictions:
                s = team_predictions['sentiment']
                report += f"- **Sentiment Distribution**:\n"
                report += f"  - Positive: {s.get('positive', 0):.1%}\n"
                report += f"  - Neutral: {s.get('neutral', 0):.1%}\n"
                report += f"  - Negative: {s.get('negative', 0):.1%}\n"

            report += "\n---\n\n"

        top_comments = self.preprocessor.filter_comments(comments)[:5]
        if top_comments:
            report += """## 🔍 Top Comments\n\n"""
            for i, c in enumerate(top_comments, 1):
                text = c.get('text', '')[:200]
                likes = c.get('like_count', 0)
                author = c.get('author', 'Anonymous')
                report += f"{i}. **{author}** ({likes} likes): {text}...\n\n"
            report += "---\n\n"

        report += """## 📌 Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- **Enhanced Prompts (v2.2)**: Two-part summary structure with KEY POINTS and DETAILED ANALYSIS
- **Enhanced Comment Analysis**: Majority/minority opinion separation with sentiment breakdown
- **LLM Category Validation**: Automated correction of misclassified categories
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.2*
"""
        return report

    def generate_report(self, video_data: Dict) -> str:
        """Generate complete report with all enhancements"""
        video_info = video_data['video_info']
        comments = video_data.get('comments', [])

        print(f"\n🎬 Processing: {video_info.get('title', 'Unknown')}")

        # Collect language detection info
        detected_languages = {}

        print("  📝 Generating video summary...")
        video_summary, video_lang = self.generate_video_summary(video_info)
        detected_languages['video'] = video_lang

        print("  💬 Generating reaction summary...")
        reaction_summary, comment_langs = self.generate_reaction_summary(
            video_info.get('title', ''), comments
        )
        detected_languages['comments'] = comment_langs

        print("  📊 Calculating metrics...")
        metrics = self.calculate_engagement_metrics(video_info)

        # Team model predictions with category correction
        team_predictions = {}

        if self.config.use_category_model:
            pred = self.team_models.predict_category(video_info)
            if pred:
                # 🆕 Apply LLM correction if enabled
                if self.config.enable_llm_category_correction:
                    print("  🔍 Validating category with LLM...")
                    original, final, corrected = self.team_models.predict_category_with_llm_correction(
                        video_info, pred
                    )
                    team_predictions['category'] = {
                        'original': original,
                        'final': final,
                        'corrected': corrected
                    }
                    if corrected:
                        print(f"    [Category corrected: {original} → {final}]")
                else:
                    team_predictions['category'] = pred

        if self.config.use_sentiment_model:
            sent = self.team_models.analyze_sentiment(comments)
            if sent:
                team_predictions['sentiment'] = sent

        print("  📄 Formatting report...")
        report = self.format_markdown_report(
            video_data, video_summary, reaction_summary,
            metrics,
            team_predictions if team_predictions else None,
            detected_languages
        )

        print("  ✅ Done!")
        return report

# Initialize report generator
report_generator = ReportGenerator(config, model_manager, preprocessor, team_models)
print("✅ Enhanced report generator initialized (v2.2)")
print("🆕 Two-part summary structure enabled")
print("🆕 Majority/minority opinion analysis enabled")
print("🆕 LLM category correction enabled")

✅ Enhanced report generator initialized (v2.2)
🆕 Two-part summary structure enabled
🆕 Majority/minority opinion analysis enabled
🆕 LLM category correction enabled


## 🚀 9. Run Enhanced Pipeline

### What's New:
- Processing up to 100 comments per video (configurable)
- Two-part structured summaries
- Majority/minority opinion separation
- LLM-validated categories (if team model enabled)

In [72]:
# Load data
dataset = data_loader.get_data()

# Select videos using config
videos_to_process = dataset[:config.num_videos_for_test]

print(f"\n🎯 Processing {len(videos_to_process)} videos...")
print(f"   (Pipeline v2.2 with enhanced prompts and analysis)")
print(f"   (Processing up to {config.max_comments_to_process} comments per video)")
print("="*60)

# Generate reports
reports = []

for i, video_data in enumerate(videos_to_process, 1):
    print(f"\n{'='*60}")
    print(f"Video {i}/{len(videos_to_process)}")
    print(f"{'='*60}")

    try:
        report = report_generator.generate_report(video_data)
        reports.append({
            'video_id': video_data['video_info']['video_id'],
            'report': report
        })
    except Exception as e:
        print(f"❌ Error: {e}")
        continue

print(f"\n{'='*60}")
print(f"✅ Completed {len(reports)}/{len(videos_to_process)} videos")
print(f"{'='*60}")


📂 Loading: /content/drive/MyDrive/25_sch/ML/youtube_test_10132154/full_dataset_20251013_215347.json
✅ Loaded 20 videos

🎯 Processing 3 videos...
   (Pipeline v2.2 with enhanced prompts and analysis)
   (Processing up to 100 comments per video)

Video 1/3

🎬 Processing: NMIXX(엔믹스) “Blue Valentine” M/V
  📝 Generating video summary...
    [Detected input language: English]
      [Input tokens: 472]
      [Output tokens: 122]
    [Output language: English]
  💬 Generating reaction summary...
    [Comment languages (top 20): {'English': 8, 'Korean': 9, 'Japanese': 3}]
      [Input tokens: 1787]
      [Output tokens: 196]
    [Output language: English]
  📊 Calculating metrics...
  📄 Formatting report...
  ✅ Done!

Video 2/3

🎬 Processing: How well do I know ROBLOX | ALBERTGUESSR
  📝 Generating video summary...
    [Detected input language: English]
      [Input tokens: 432]
      [Output tokens: 160]
    [Output language: English]
  💬 Generating reaction summary...
    [Comment languages (to

## 📊 10. Display Reports

In [73]:
from IPython.display import Markdown, display

for i, report_data in enumerate(reports, 1):
    print(f"\n{'='*80}")
    print(f"REPORT {i}/{len(reports)} - Video ID: {report_data['video_id']}")
    print(f"{'='*80}\n")
    display(Markdown(report_data['report']))
    print("\n" + "="*80 + "\n")


REPORT 1/3 - Video ID: EmeW6li6bbo



# YouTube Video Report

**Generated**: 2025-12-11 12:22:08
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.2 (Enhanced Prompts & Analysis)

---

## 📹 Video Information

- **Title**: NMIXX(엔믹스) “Blue Valentine” M/V
- **Channel**: JYP Entertainment
- **Category**: Music
- **Published**: 2025-10-13T09:00:02Z
- **Duration**: 195 seconds
- **Video ID**: `EmeW6li6bbo`
- **URL**: https://www.youtube.com/watch?v=EmeW6li6bbo

### 🌍 Detected Languages

- **Video content**: English
- **Comments**: English: 8, Korean: 9, Japanese: 3
- **Report output**: English

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 1,295,943 |
| Likes | 66,047 |
| Comments | 6,284 |
| Engagement Rate | 5.58% |
| Like Rate | 5.1% |
| Comment Rate | 0.48% |

---

## 📝 Video Summary

KEY POINT:
The music video for NMIXX's song "Blue Valentine" showcases the group's energetic and emotive performance.

DETAILED SUMMARY:
NMIXX's "Blue Valentine" music video primarily focuses on the group's dynamic performance, with a mix of dark and vibrant visuals. The video seems to explore themes of love and relationships, with the group members conveying a range of emotions. Targeting K-pop fans, the video is likely intended to promote the song and the group's upcoming activities. Unfortunately, the description does not provide further context, so some aspects of the video remain unclear.

---

## 💬 Audience Reaction Summary

**Comments Analyzed**: 20 / 20 (100.0%)

KEY POINT:
The audience reaction to NMIXX's "Blue Valentine" music video is overwhelmingly positive excitement, with fans praising the group's unique concept, music quality, and emotional delivery.

DETAILED SUMMARY:
The majority of comments express deep admiration for NMIXX's growth and uniqueness, with many fans highlighting the song's nostalgic and dreamy vibe. The emotional delivery of the lyrics and the vocal performances are also widely praised, with some fans even considering this song a highlight of their career. A small minority of comments, mostly in Japanese, focus on the song's catchy melody and NMIXX's stylish vocals. There are also a few comments that appreciate the song's themes of universal love and its avoidance of being consumed by the idol industry. Cultural and language differences are not prominent in this analysis, as the fans' reactions are largely uniform across languages.

SENTIMENT BREAKDOWN:
Positive: 95%  Negative: 2%  Neutral: 3%

Note: This sentiment analysis is based on the top 20 high‑engagement comments and may not fully represent the entire audience.

---
## 🔍 Top Comments

1. **@kyujified** (806 likes): I get really emotional thinking about how far NMIXX have come. From their debut with O.O, when so many people misunderstood their concept and doubted them, to now, where everyone finally sees their ta...

2. **@nothing-mt7kv** (691 likes): 와 진심 모든 면에서 레전드다 노래 퀄리티 보컬 컨셉 엔믹스느낌 대중성 뮤비 안무 비주얼 전부 완벽하다 진짜 너무 좋다ㅠㅠ...

3. **@alejandromoreno1111** (599 likes): Congratulations to NMIXX for being the only group to have Lily, Haewon, Bae, Sullyoon, Jiwoo and Kyujin as its members. Total privilege...

4. **@Laenyu** (391 likes): 선공개 곡부터 노래가 진행되면서 가슴에 벅참과 감동이 느껴졌었고 블루 발렌타인이 그 기분을 또 다시 느끼게 해줬음
진정 노래를 사랑하고 즐기는 사람들이 존재한다면 이 노래는 필히 유행한다......

5. **@rynnieeefairy** (320 likes): this song feels so upbeat and nostalgic!! and like many are saying, their sound/color is so unique, i'm so happy that they're always staying true to their uniqueness!! nmixx and nswer lets make this s...

---

## 📌 Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- **Enhanced Prompts (v2.2)**: Two-part summary structure with KEY POINTS and DETAILED ANALYSIS
- **Enhanced Comment Analysis**: Majority/minority opinion separation with sentiment breakdown
- **LLM Category Validation**: Automated correction of misclassified categories
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.2*





REPORT 2/3 - Video ID: RGbDW-hyxxc



# YouTube Video Report

**Generated**: 2025-12-11 12:22:34
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.2 (Enhanced Prompts & Analysis)

---

## 📹 Video Information

- **Title**: How well do I know ROBLOX | ALBERTGUESSR
- **Channel**: Flamingo
- **Category**: Gaming
- **Published**: 2025-10-13T03:13:19Z
- **Duration**: 1630 seconds
- **Video ID**: `RGbDW-hyxxc`
- **URL**: https://www.youtube.com/watch?v=RGbDW-hyxxc

### 🌍 Detected Languages

- **Video content**: English
- **Comments**: English: 20
- **Report output**: English

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 349,586 |
| Likes | 23,507 |
| Comments | 2,703 |
| Engagement Rate | 7.5% |
| Like Rate | 6.72% |
| Comment Rate | 0.77% |

---

## 📝 Video Summary

KEY POINT:
This YouTube video features a gameplay challenge where the host, AlbertGUESSR, tries to guess various Roblox games he is teleported into, with the goal of identifying as many games as possible within a set time limit.

DETAILED SUMMARY:
The video showcases a mix of easy, medium, hard, and impossible mode games, with links provided to the games in the description. AlbertGUESSR's knowledge of Roblox games is tested through a GeoGuessr-style gameplay experience, where he is randomly placed into different games. The target audience appears to be fans of Roblox and gaming enthusiasts, as the content is centered around guessing and identifying Roblox games. The purpose of the video seems to be entertainment and community engagement, with a dash of friendly competition.

---

## 💬 Audience Reaction Summary

**Comments Analyzed**: 20 / 20 (100.0%)

KEY POINT:
The audience is overwhelmingly excited and nostalgic for the ROBLOX series, particularly the return of Kaden.

DETAILED SUMMARY:
The majority of comments (over 90%) are positive, with viewers expressing their enthusiasm for the series and the return of Kaden. Many fans are thrilled to see their favorite YouTubers reunited and are sharing memories of the old collabs. A small minority of comments (around 5%) express disappointment or sadness about the series' past, but these views are largely drowned out by the excitement. Notably, some fans are also appreciative of Albert's efforts to support smaller creators, as seen in comments praising his collaborations with Flamingo. Overall, the comments showcase a strong sense of community and shared nostalgia among ROBLOX fans.

SENTIMENT BREAKDOWN:
Positive: 92%  Negative: 4%  Neutral: 4%

Note: This sentiment analysis is based on the top 20 high‑engagement comments and may not fully represent the entire audience.

---
## 🔍 Top Comments

1. **@klippyyyy** (2430 likes): I want a divorce, Albert....

2. **@ilikecats382** (2283 likes): Yes we love to see Kaden back...

3. **@Emizsocool** (1325 likes): KADEN IS OUT OF PRISON, YAYYYYYYYY...

4. **@aster-o5f** (919 likes): ANOTHER 30 MIN VID?? WE ARE BEING BLESSED "THANK YOU ALBERT" WE ALL SAY IN UNISON...

5. **@Loveforsara** (697 likes): 0:10 KADEN IS BACK??!!! OMG OMGG...

---

## 📌 Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- **Enhanced Prompts (v2.2)**: Two-part summary structure with KEY POINTS and DETAILED ANALYSIS
- **Enhanced Comment Analysis**: Majority/minority opinion separation with sentiment breakdown
- **LLM Category Validation**: Automated correction of misclassified categories
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.2*





REPORT 3/3 - Video ID: ux3QETpLcPs



# YouTube Video Report

**Generated**: 2025-12-11 12:23:02
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.2 (Enhanced Prompts & Analysis)

---

## 📹 Video Information

- **Title**: 劇場版『チェンソーマン レゼ篇』オープニングムービー 　主題歌：米津玄師「IRIS OUT」“Chainsaw Man – The Movie: Reze Arc” – Opening Movie
- **Channel**: Kenshi Yonezu  米津玄師
- **Category**: Music
- **Published**: 2025-10-07T10:59:56Z
- **Duration**: 151 seconds
- **Video ID**: `ux3QETpLcPs`
- **URL**: https://www.youtube.com/watch?v=ux3QETpLcPs

### 🌍 Detected Languages

- **Video content**: Japanese
- **Comments**: Japanese: 19, English: 1
- **Report output**: English

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 9,096,175 |
| Likes | 416,328 |
| Comments | 14,691 |
| Engagement Rate | 4.74% |
| Like Rate | 4.58% |
| Comment Rate | 0.16% |

---

## 📝 Video Summary

KEY POINT:
This video is the opening movie for the theatrical anime adaptation of the popular manga "Chainsaw Man" and features the theme song "IRIS OUT" by Kenshi Yonezu.

DETAILED SUMMARY:
The video showcases the main theme of the "Chainsaw Man" story, which revolves around the protagonist Denji, a young devil hunter who forms a close bond with a girl named Makima. As Denji's life changes after meeting Makima, the video highlights the story's blend of action, romance, and fantasy elements. The video is likely targeting fans of the original manga series and anime adaptation, as well as new viewers interested in the story's unique blend of genres. The main trailer features a mix of action-packed sequences and emotional moments, setting the tone for the upcoming theatrical anime.

---

## 💬 Audience Reaction Summary

**Comments Analyzed**: 20 / 20 (100.0%)

KEY POINT:
The audience reaction is overwhelmingly positive excitement, with fans praising the opening movie's engaging storyline, lovable characters, and impressive visuals.

DETAILED SUMMARY:
The majority of comments (over 90%) express admiration for the opening movie, highlighting its ability to capture the audience's attention and set the tone for a thrilling story. Fans appreciate the development of the main characters, particularly Denji and Akane, and their relationships with each other and other supporting characters. Many commenters also praise the animation, music, and overall production quality, comparing it to a "must-watch" experience. A small minority (around 5%) of comments express enthusiasm for the movie's free availability, but this is not a dominant theme. Notably, some commenters mention the opening's ability to evoke strong emotions, such as happiness and nostalgia.

SENTIMENT BREAKDOWN:
Positive: 95%  Negative: 2%  Neutral: 3%

Note: This sentiment analysis is based on the top 20 high‑engagement comments and may not fully represent the entire audience.

---
## 🔍 Top Comments

1. **@YAMIGMISAMA** (29539 likes): 映画館でこのムービーと一緒に大音量で曲が流れた時の幸福感エグかった...

2. **@summer-xf2zo** (21043 likes): レゼ編はマジで、OPで「始まるぞ！！」って感じて、EDで「終わった......。」って感じられる最高の作品。...

3. **@漢梅トレイン** (18128 likes): 1:05 ここのポチタダンスまじで可愛すぎる...

4. **@ひろのらごん** (12483 likes): YouTubeで聴くのも良いけど、映画館で聴いたときほんとに鳥肌たった。OP聴いただけで見に来て良かったってなった...

5. **@ニコち-y6k** (9829 likes): 0:54 2:05ボンやバンがないのまだ生活の中にレゼがいない感じ好き…...

---

## 📌 Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- **Enhanced Prompts (v2.2)**: Two-part summary structure with KEY POINTS and DETAILED ANALYSIS
- **Enhanced Comment Analysis**: Majority/minority opinion separation with sentiment breakdown
- **LLM Category Validation**: Automated correction of misclassified categories
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.2*


## 📈 11. Language Statistics

In [29]:
stats = report_generator.get_language_statistics()

print("\n" + "="*60)
print("Language Statistics (Enhanced Processing)")
print("="*60)

print("\n📹 Video Languages:")
for lang, count in stats['video'].items():
    print(f"  - {lang}: {count} videos")

print("\n💬 Comment Languages (sampled from top comments):")
for lang, count in stats['comments'].items():
    print(f"  - {lang}: {count} comments")

print("\n🎯 Processing Summary:")
print(f"  - Comments per video: up to {config.max_comments_to_process}")
print(f"  - Total videos processed: {len(reports)}")
print(f"  - Output language: {config.output_language} (fixed)")

print("\n" + "="*60)


Language Statistics (Enhanced Processing)

📹 Video Languages:
  - English: 2 videos
  - Japanese: 1 videos

💬 Comment Languages (sampled from top comments):
  - English: 29 comments
  - Korean: 9 comments
  - Japanese: 22 comments

🎯 Processing Summary:
  - Comments per video: up to 100
  - Total videos processed: 3
  - Output language: English (fixed)



## 💾 12. Save Reports (Optional)

In [74]:
if config.save_reports:
    import os
    from pathlib import Path

    # Create output directory
    output_dir = Path(config.output_dir)
    output_dir.mkdir(exist_ok=True)

    # Save individual reports
    for report_data in reports:
        video_id = report_data['video_id']
        filepath = output_dir / f"report_{video_id}_v2.2.md"

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(report_data['report'])

        print(f"✅ Saved: {filepath}")

    # Save summary
    summary_path = output_dir / f"summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}_v2.2.json"
    summary_data = {
        'pipeline_version': '2.2',
        'model': config.model_name,
        'videos_processed': len(reports),
        'max_comments_per_video': config.max_comments_to_process,
        'language_stats': stats,
        'enhancements': [
            'Two-part summary structure',
            'Majority/minority opinion analysis',
            'Sentiment breakdown integration',
            'LLM category correction'
        ]
    }

    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Summary saved: {summary_path}")
    print(f"\n📁 All reports saved to: {output_dir.absolute()}")

✅ Saved: reports/report_EmeW6li6bbo_v2.2.md
✅ Saved: reports/report_RGbDW-hyxxc_v2.2.md
✅ Saved: reports/report_ux3QETpLcPs_v2.2.md

✅ Summary saved: reports/summary_20251211_122541_v2.2.json

📁 All reports saved to: /content/reports


In [75]:
import shutil
import os

source = '/content/reports_2512112125'
target = '/content/drive/MyDrive/25_sch/ML/results'

# 대상 디렉토리 생성 (없을 경우)
os.makedirs(target, exist_ok=True)

# 폴더 이동
shutil.move(source, target)
print(f"폴더가 {target} 으로 이동되었습니다.")

폴더가 /content/drive/MyDrive/25_sch/ML/results 으로 이동되었습니다.


## 🌟 13. Real-Time YouTube URL Report Generator

In [84]:
import re
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import time

def extract_video_id(url: str) -> str:
    """Extract video_id from YouTube URL

    Supports formats:
    - https://www.youtube.com/watch?v=VIDEO_ID
    - https://youtu.be/VIDEO_ID
    - https://www.youtube.com/embed/VIDEO_ID
    """
    patterns = [
        r'(?:v=|\/)([0-9A-Za-z_-]{11}).*',
        r'(?:embed\/)([0-9A-Za-z_-]{11})',
        r'(?:\/watch\?v=)([0-9A-Za-z_-]{11})'
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    raise ValueError("Invalid YouTube URL format")

def fetch_video_data(youtube, video_id: str, max_results=100) -> Dict:
    """Fetch video info and comments from YouTube API

    Args:
        youtube: YouTube API client
        video_id: Video ID to fetch
        max_results: Maximum number of comments to collect (default: 100)

    Returns:
        Dict with 'video_info' and 'comments' keys
    """
    try:
        # Fetch video information
        print(f"  📹 Fetching video information...")
        video_response = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=video_id
        ).execute()

        if not video_response['items']:
            raise ValueError(f"Video not found: {video_id}")

        video_item = video_response['items'][0]
        video_info = {
            'video_id': video_id,
            'title': video_item['snippet']['title'],
            'channel_title': video_item['snippet']['channelTitle'],
            'category_name': video_item['snippet'].get('categoryId', 'N/A'),
            'published_at': video_item['snippet']['publishedAt'],
            'description': video_item['snippet']['description'],
            'view_count': int(video_item['statistics'].get('viewCount', 0)),
            'like_count': int(video_item['statistics'].get('likeCount', 0)),
            'comment_count': int(video_item['statistics'].get('commentCount', 0)),
            'duration': video_item['contentDetails']['duration']
        }
        print(f"  ✅ Video: {video_info['title'][:50]}...")

        # Fetch comments (up to max_results)
        comments = []
        next_page_token = None

        print(f"  💬 Fetching comments (max: {max_results})...")
        while len(comments) < max_results:
            comment_response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=min(100, max_results - len(comments)),
                pageToken=next_page_token
            ).execute()

            for item in comment_response['items']:
                top_comment = item['snippet']['topLevelComment']['snippet']
                comments.append({
                    'text': top_comment['textDisplay'],
                    'like_count': top_comment.get('likeCount', 0),
                    'author': top_comment['authorDisplayName']
                })

            next_page_token = comment_response.get('nextPageToken')
            if not next_page_token:
                break

        video_data = {
            'video_info': video_info,
            'comments': comments[:max_results]
        }

        print(f"  ✅ Collected {len(comments)} comments")
        return video_data

    except HttpError as e:
        print(f"  ❌ YouTube API error: {e}")
        raise
    except Exception as e:
        print(f"  ❌ Data collection failed: {e}")
        raise

def generate_report_from_url(youtube_url: str, youtube_api_key: str = None,
                             max_comments: int = 100) -> Optional[str]:
    """Generate report from YouTube URL in real-time

    Args:
        youtube_url: YouTube video URL
        youtube_api_key: YouTube Data API key (get from Google Cloud Console)
        max_comments: Maximum comments to fetch (default: 100)

    Returns:
        Generated markdown report or None if failed

    Example:
        >>> report = generate_report_from_url(
        ...     "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
        ...     YOUR_API_KEY
        ... )
    """
    print("=" * 80)
    print("🚀 Real-Time YouTube Report Generation")
    print("=" * 80)

    # Extract Video ID
    try:
        video_id = extract_video_id(youtube_url)
        print(f"📺 Video ID: {video_id}")
    except ValueError as e:
        print(f"❌ Error: {e}")
        return None

    # Initialize YouTube API
    if not youtube_api_key:
        print("⚠️  Warning: No API key provided")
        print("   Get your API key from: https://console.developers.google.com")
        return None

    youtube = build('youtube', 'v3', developerKey=youtube_api_key)

    try:
        # Fetch data from YouTube
        print("\n📥 Fetching data from YouTube API...")
        video_data = fetch_video_data(youtube, video_id, max_comments)

        # Generate report using existing pipeline
        print("\n🔄 Generating report with LLM pipeline...")
        report = report_generator.generate_report(video_data)

        # Display result
        print("\n" + "=" * 80)
        print("✅ REPORT GENERATION COMPLETED")
        print("=" * 80)

        from IPython.display import display, Markdown
        display(Markdown(report))

        return report

    except Exception as e:
        print(f"\n❌ Report generation failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# ============================================================
# Usage Instructions
# ============================================================

print("""
📝 How to Use Real-Time Report Generator:

1. Get YouTube Data API Key:
   - Visit: https://console.developers.google.com
   - Create project and enable YouTube Data API v3
   - Create credentials (API Key)

2. Set your API key:
   YOUR_YOUTUBE_API_KEY = "paste_your_api_key_here"

3. Generate report:
   report = generate_report_from_url(
       "https://www.youtube.com/watch?v=VIDEO_ID",
       YOUR_YOUTUBE_API_KEY,
       max_comments=100
   )

⚠️  Note: YouTube API has daily quota limits (10,000 units/day)
   Each report generation uses approximately 100-150 units
""")

# ============================================================
# Example Usage (uncomment and add your API key to test)
# ============================================================

# YOUR_YOUTUBE_API_KEY = ""  # Add your API key here
#
# # Example: Generate report from URL
# report = generate_report_from_url(
#     "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
#     YOUR_YOUTUBE_API_KEY,
#     max_comments=100
# )



📝 How to Use Real-Time Report Generator:

1. Get YouTube Data API Key:
   - Visit: https://console.developers.google.com
   - Create project and enable YouTube Data API v3
   - Create credentials (API Key)

2. Set your API key:
   YOUR_YOUTUBE_API_KEY = "paste_your_api_key_here"

3. Generate report:
   report = generate_report_from_url(
       "https://www.youtube.com/watch?v=VIDEO_ID",
       YOUR_YOUTUBE_API_KEY,
       max_comments=100
   )

⚠️  Note: YouTube API has daily quota limits (10,000 units/day)
   Each report generation uses approximately 100-150 units



In [85]:
def generate_report_from_url(youtube_url: str, youtube_api_key: str = None, max_comments=100):
    """URL 입력 → 즉시 Report 생성"""
    print("=" * 80)
    print("🚀 실시간 YouTube Report 생성")
    print("=" * 80)

    # Video ID 추출
    video_id = extract_video_id(youtube_url)
    print(f"📺 Video ID: {video_id}")

    # YouTube API 초기화
    if not youtube_api_key:
        print("⚠️  API 키 없음 - 데모 데이터로 대체")
        return "YouTube API 키를 입력하세요."

    youtube = build('youtube', 'v3', developerKey=youtube_api_key)

    try:
        # 데이터 수집
        video_data = fetch_video_data(youtube, video_id, max_comments)

        # 기존 파이프라인으로 Report 생성
        report = report_generator.generate_report(video_data)

        # 결과 출력
        print("\n" + "=" * 80)
        print("✅ REPORT COMPLETED")
        print("=" * 80)
        from IPython.display import display, Markdown
        display(Markdown(report))

        return report

    except Exception as e:
        print(f"❌ Report 생성 실패: {e}")
        return None

In [ ]:
# ====================
# 사용법 (API 키 입력 후 사용)
# ====================
YOUR_YOUTUBE_API_KEY = ""  # https://console.developers.google.com에서 발급

# 사용 예시
report = generate_report_from_url(
    "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    YOUR_YOUTUBE_API_KEY
)

🚀 실시간 YouTube Report 생성
📺 Video ID: dQw4w9WgXcQ
  📹 Fetching video information...
  ✅ Video: Rick Astley - Never Gonna Give You Up (Official Vi...
  💬 Fetching comments (max: 100)...
  ✅ Collected 100 comments

🎬 Processing: Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster)
  📝 Generating video summary...
    [Detected input language: English]
      [Input tokens: 815]
      [Output tokens: 147]
    [Output language: English]
  💬 Generating reaction summary...
    [Comment languages (top 97): {'English': 97}]
      [Input tokens: 2694]
      [Output tokens: 220]
    [Output language: English]
  📊 Calculating metrics...
  📄 Formatting report...
  ✅ Done!

✅ REPORT COMPLETED


# YouTube Video Report

**Generated**: 2025-12-11 12:50:56
**Model**: meta-llama/Llama-3.1-8B-Instruct
**Pipeline Version**: 2.2 (Enhanced Prompts & Analysis)

---

## 📹 Video Information

- **Title**: Rick Astley - Never Gonna Give You Up (Official Video) (4K Remaster)
- **Channel**: Rick Astley
- **Category**: 10
- **Published**: 2009-10-25T06:57:33Z
- **Duration**: 214 seconds
- **Video ID**: `dQw4w9WgXcQ`
- **URL**: https://www.youtube.com/watch?v=dQw4w9WgXcQ

### 🌍 Detected Languages

- **Video content**: English
- **Comments**: English: 97
- **Report output**: English

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | 1,721,242,047 |
| Likes | 18,672,714 |
| Comments | 2,408,603 |
| Engagement Rate | 1.22% |
| Like Rate | 1.08% |
| Comment Rate | 0.14% |

---

## 📝 Video Summary

KEY POINT:
This official music video is a remastered version of Rick Astley's hit song "Never Gonna Give You Up," released in 1987.

DETAILED SUMMARY:
The video showcases the catchy and upbeat tune, featuring Rick Astley singing and performing the song with his iconic style. The song, written and produced by Stock Aitken and Waterman, was a global success, topping the charts in 25 countries, including the US Billboard Hot 100. The video, directed by Simon West, has become a legendary piece, passing the 1 billion YouTube views milestone in 2021. The song's lyrics, which emphasize commitment and loyalty, have become a classic and are still widely popular today.

---

## 💬 Audience Reaction Summary

**Comments Analyzed**: 97 / 100 (97.0%)

KEY POINT:
Overwhelmingly positive excitement and nostalgia dominate the comments, as fans celebrate the iconic song and share their experiences of being "Rickrolled."

DETAILED SUMMARY:
The majority of comments express enthusiasm and fondness for the song, with many users sharing their personal stories of being "Rickrolled" and how it made them laugh or smile. A notable minority of comments, mostly in English, express frustration or annoyance at being misled, but these are largely outweighed by the positive reactions. The comments also reveal a cultural phenomenon, with the term "Rickroll" becoming a shared experience and a symbol of internet culture. Language differences are minimal, with English comments making up the majority, but Korean and Japanese comments also express similar sentiments of nostalgia and excitement. Code-switching comments, such as those in Korean and English, demonstrate the fluidity of language in online communities.

SENTIMENT BREAKDOWN:
Positive: 85%  Negative: 10%  Neutral: 5%

(Exact percentages might vary based on manual sentiment analysis, but this gives a general idea of the sentiment distribution.)

---
## 🔍 Top Comments

1. **@YouTube** (145745 likes): can confirm: he never gave us up...

2. **@TrầnBảoVinh-l6p** (8 likes): I don&#39;t play this song to get rickrolled, I play this song to enjoy it...

3. **@Flutters1211** (5 likes): “As long as the trolls never stop trolling, the Rick will never stop rolling.”<br>- cotter548...

4. **@EviKrimson** (4 likes): Well this is not the link for free bitcoin...

5. **@gaming777official-army** (3 likes): Anyone in 2026👉👈...

---

## 📌 Technical Notes

- This report was automatically generated using LLM-based multilingual analysis
- **Enhanced Prompts (v2.2)**: Two-part summary structure with KEY POINTS and DETAILED ANALYSIS
- **Enhanced Comment Analysis**: Majority/minority opinion separation with sentiment breakdown
- **LLM Category Validation**: Automated correction of misclassified categories
- Input content processed in original language(s) without translation layer
- Output language fixed to English for consistency

---

*Generated by YouTube Report Generator - Phase 2 Full Pipeline v2.2*


## 🎓 Summary of Improvements (v2.2)

### 1️⃣ Summary Prompt Tuning
- ✅ **Two-Part Structure**: KEY POINT (1-2 sentences) + DETAILED SUMMARY (3-5 sentences)
- ✅ **Improved Scannability**: Users can quickly grasp main points
- ✅ **Better LLM Output Quality**: Structured format reduces verbosity

### 2️⃣ Enhanced Comment Analysis
- ✅ **Increased Comment Processing**: 50 → 100 comments (configurable)
- ✅ **Majority/Minority Separation**: Distinguishes 80%+ views from outliers
- ✅ **Sentiment Breakdown**: Explicit percentages (Positive/Negative/Neutral)
- ✅ **Outlier Filtering**: Ignores absurd comments (<5%)

### 3️⃣ LLM Category Correction
- ✅ **Automated Validation**: LLM reviews model predictions
- ✅ **Correction Tracking**: Shows original → final category if corrected
- ✅ **Configurable**: Can be enabled/disabled in config

### 4️⃣ Report Format Improvements
- ✅ **Structured Sections**: Clear separation of KEY POINTS and DETAILED ANALYSIS
- ✅ **Enhanced Metrics**: Shows comments analyzed vs total
- ✅ **Category Status**: Displays validation/correction status

---

## 🔄 Next Steps

### Immediate Testing
1. Run on test videos and evaluate quality improvements
2. Compare v2.1 vs v2.2 outputs side-by-side
3. Gather feedback on new two-part structure

### Future Enhancements (Not Implemented)
- Token efficiency optimization (dynamic `max_new_tokens`)
- Quality gate with regeneration for poor summaries
- Multi-GPU support for faster processing
- Real-time monitoring with WandB/Prometheus

---

*Pipeline v2.2 - Enhanced Prompts & Analysis*